In [2]:
import numpy as np
import pandas as pd

import gensim
import nltk
import stanza

## Downloads

In [3]:
import gensim.downloader as gensim_api
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
from nltk.corpus import wordnet

nltk.download('omw-1.4')

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse,ner')
wv = gensim_api.load('fasttext-wiki-news-subwords-300')
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
sentiment_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/brianwilliams/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


2022-08-16 13:17:46 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-08-16 13:17:48 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

2022-08-16 13:17:48 INFO: Use device: cpu
2022-08-16 13:17:48 INFO: Loading: tokenize
2022-08-16 13:17:48 INFO: Loading: pos
2022-08-16 13:17:48 INFO: Loading: lemma
2022-08-16 13:17:48 INFO: Loading: depparse
2022-08-16 13:17:48 INFO: Loading: ner
2022-08-16 13:17:48 INFO: Done loading processors!


## Load and preprocess data

In [24]:
sheet_id = "1Bk426YSho-npLlcpt8zTwuiVosQtGeMMJ_hj3rqZAqA"
sheet_name = "main"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

goals_df = pd.read_csv(url)
goals = goals_df["Goal"].copy()
goals = goals.str.translate(str.maketrans('', '', '.,'))
goals

0                      Blog
1               Lose weight
2              Write a book
3      Stop procrastinating
4              Fall in love
               ...         
599         earn more money
600                 study  
601    get better at makeup
602               be pretty
603         start investing
Name: Goal, Length: 604, dtype: object

## Sentence embeddings

In [25]:
SIMILARITY_THRESHOLD = 0.5

goal_embeddings = sentence_model.encode(goals, convert_to_tensor=True)
cosine_scores = util.cos_sim(goal_embeddings, goal_embeddings)

In [26]:
# Rank all similarities between existing goals
goal_similarities = pd.DataFrame(columns=["goal_x", "goal_y", "similarity"])

for i in range(len(goal_embeddings) - 1):
    for j in range(i + 1, len(goal_embeddings) - 1):
        if SIMILARITY_THRESHOLD < cosine_scores[i][j] < 0.99:
            row = pd.DataFrame({
                "goal_x": goals[i],
                "goal_y": goals[j],
                "similarity": float(cosine_scores[i][j])
                }, index=[0])
            goal_similarities = pd.concat([goal_similarities, row])

goal_similarities = goal_similarities.sort_values(by="similarity", ascending=False).reset_index(drop=True)
goal_similarities.head()

,goal_x,goal_y,similarity
0,Practice gratitude everyday,Practice gratitude daily,0.989245
1,Learn to cook,Learn how to cook,0.976222
2,Learn how to cook,learn to cook,0.976222
3,get 8 hours of sleep each night,get eight hours of sleep every night,0.966550
4,get better at communicating,be better at communicating,0.962177


## Dependency parsing

In [27]:
def extract_root(phrase):
  roots = [word.lemma for sent in nlp(phrase).sentences for word in sent.words if word.deprel == 'root']
  return roots[0] if len(roots) > 0 else None

def extract_words(phrase):
  words = [word for sent in nlp(phrase).sentences for word in sent.words]

  return words

In [28]:
def extract_important(phrase, asdict=False):
  words = [word for sent in nlp(phrase).sentences for word in sent.words]

  roots = [word.lemma for word in filter(lambda w: w.deprel == 'root', words)]
  xcomps = [word.lemma for word in filter(lambda w: w.deprel == 'xcomp', words)]
  quantities = [{
    'item': words[word.head - 1].text,
    'amount': word.lemma
    } for word in filter(lambda w: w.deprel == 'nummod', words)]

  root = roots[0] if len(roots) > 0 else None

  important = {
    'root': root,
    'xcomps': xcomps,
    'quantities': quantities
  }
  return list(important.values()) if not asdict else important

extract_important('Do 12 pushups every three days', asdict=True)

{'root': 'do',
 'xcomps': [],
 'quantities': [{'item': 'pushups', 'amount': '12'},
  {'item': 'days', 'amount': 'three'}]}

In [29]:
goals_df[['root', 'xcomps', 'quantities']] = pd.DataFrame(goals.transform(extract_important).tolist(), index=goals_df.index)
goals_df

,Goal,Source,Style,Type,root,Unnamed: 5,Unnamed: 6,xcomps,quantities
0,Blog,NaN,Practice,NaN,blog,NaN,NaN,[],[]
1,Lose weight,NaN,Lifestyle,NaN,lose,NaN,NaN,[],[]
2,Write a book,NaN,Target,NaN,write,NaN,NaN,[],[]
3,Stop procrastinating,NaN,Lifestyle,NaN,stop,NaN,NaN,[],[]
4,Fall in love,NaN,Target,NaN,fall,NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...
599,earn more money,NaN,NaN,NaN,earn,NaN,NaN,[],[]
600,study,NaN,NaN,NaN,study,NaN,NaN,[],[]
601,get better at makeup,NaN,NaN,NaN,get,NaN,NaN,[good],[]
602,be pretty,NaN,NaN,NaN,be,NaN,NaN,[],[]


In [30]:
def get_antonyms(word):
    antonyms = set()

    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            if l.antonyms():
                antonyms.add(l.antonyms()[0].name())
    
    return antonyms

In [31]:
get_antonyms('gain')

{'fall_back', 'lose', 'loss', 'reduce'}

# Sentiment (currently unused)

In [32]:
goals_df[['sentiment', 'sentiment_confidence']] = pd.DataFrame(goals_df['Goal'].transform(lambda x: list(sentiment_classifier(x)[0].values())).tolist(), index=goals_df.index)
goals_df

,Goal,Source,Style,Type,root,Unnamed: 5,Unnamed: 6,xcomps,quantities,sentiment,sentiment_confidence
0,Blog,NaN,Practice,NaN,blog,NaN,NaN,[],[],POSITIVE,0.788019
1,Lose weight,NaN,Lifestyle,NaN,lose,NaN,NaN,[],[],NEGATIVE,0.999560
2,Write a book,NaN,Target,NaN,write,NaN,NaN,[],[],POSITIVE,0.996259
3,Stop procrastinating,NaN,Lifestyle,NaN,stop,NaN,NaN,[],[],NEGATIVE,0.998419
4,Fall in love,NaN,Target,NaN,fall,NaN,NaN,[],[],POSITIVE,0.998757
...,...,...,...,...,...,...,...,...,...,...,...
599,earn more money,NaN,NaN,NaN,earn,NaN,NaN,[],[],POSITIVE,0.994392
600,study,NaN,NaN,NaN,study,NaN,NaN,[],[],POSITIVE,0.993381
601,get better at makeup,NaN,NaN,NaN,get,NaN,NaN,[good],[],NEGATIVE,0.893811
602,be pretty,NaN,NaN,NaN,be,NaN,NaN,[],[],POSITIVE,0.999698


In [33]:
goals_df.to_csv('data/goals_df.csv', index=False)

## Most similar

In [34]:
def wv_similarity(word1, word2):
  try:
    return wv.similarity(word1, word2)
  except:
    return 0

def most_similar_goal(goal):
  goal_encoded = sentence_model.encode(goal, convert_to_tensor=True)
  similarities = util.cos_sim(goal_encoded, goal_embeddings)[0]
  return (goals[np.argmax(similarities).item()], max(similarities).item())


def most_similar_goals(goal):
  goal_encoded = sentence_model.encode(goal, convert_to_tensor=True)
  similarities = util.cos_sim(goal_encoded, goal_embeddings)[0]

  root = extract_root(goal)
  root_similarities = pd.Series([wv_similarity(root, r) for r in goals_df['root']])
  has_root_antonym = goals_df['root'].isin(get_antonyms(root))
  
  return pd.DataFrame({
    "goals": goals,
    "similarities": similarities,
    "root_similarities": root_similarities,
    "has_root_antonym": has_root_antonym,
  }).sort_values(by='similarities', ignore_index=True, ascending=False)

In [35]:
most_similar_goals('get elected president').head(20)

,goals,similarities,root_similarities,has_root_antonym
0,get elected president,1.000000,1.000000,False
1,become president,0.838123,0.590773,False
2,get elected to a government position,0.710124,0.468722,False
3,get elected to the senate,0.635445,0.468722,False
4,start following politics,0.492084,0.603532,False
5,Become a better leader,0.418541,0.590773,False
6,win an award,0.399529,0.565948,False
7,Get a job,0.397505,1.000000,False
8,get a job,0.397505,1.000000,False
9,become first chair,0.395770,0.590773,False
